<a href="https://colab.research.google.com/github/kaispace30098/Keras-rl_DQN/blob/main/DQN_Agent_by_Keras_rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create a deep learning model to solve a OpenAI Gym Lunar Lander.
0.Open the Lunar Lander environment from gym
1.Build the deep learning model by keras Sequential API with Embedding and Dense layers
2.Import the Epsilon Greedy policy and Sequential Memory deque from keras-rl2's rl
3.input the model, policy, and the memory in to rl.agent.DQNAgent and compile the model and fit
4.using wrappers to record videos to check the quaily of the training: agent.test(env,nb_episodes=10,visualize=False)
5.Save the model's weight

In [1]:
#import environment and visualization

!pip install gym[box2d]
import gym
from gym import wrappers
!pip install gym[classic_control]

#import Deep Learning api
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Input, Embedding,Reshape
from tensorflow.keras.optimizers import Adam

#import rl agent library
!pip install gym
!pip install keras
!pip install keras-rl2

#data manipulation
import numpy as np
import pandas as pd
import random 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#gym.envs.register(
#    id='cartpole-challenge',
#    entry_point='gym.envs.classic_control:CartPoleEnv',
#    max_episode_steps=5000,      
#     reward_threshold=-110.0,
#)

env = gym.make('LunarLander-v2')
#Random Search and view outcome
EPISODES=100

for i in range(EPISODES):
  return_=0
  env._max_episode_steps=5000
  env.reset()
  
  done=False
  while not done:
    action=env.action_space.sample()
    next_step,reward,done,_=env.step(action)
    return_+=reward
  print('at episode',i,', the return is ', return_)



/usr/local/lib/python3.8/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


at episode 0 , the return is  -87.9124891069432
at episode 1 , the return is  -61.186994493582446
at episode 2 , the return is  -395.82502185102015
at episode 3 , the return is  -80.07989133906808
at episode 4 , the return is  -106.69185241398156
at episode 5 , the return is  -189.5632533948967
at episode 6 , the return is  -105.37527660644952
at episode 7 , the return is  -114.77572594276735
at episode 8 , the return is  -122.5629053002499
at episode 9 , the return is  -412.1977271616926
at episode 10 , the return is  -228.59100445957958
at episode 11 , the return is  -127.83941447237864
at episode 12 , the return is  -156.43044895880726
at episode 13 , the return is  -130.4715157482966
at episode 14 , the return is  -382.51592683510273
at episode 15 , the return is  -145.22039247743737
at episode 16 , the return is  -171.2439696259081
at episode 17 , the return is  -164.06977386679546
at episode 18 , the return is  -180.60015306869644
at episode 19 , the return is  -313.7815467410087

In [3]:
actions=env.action_space.n
states=env.observation_space.shape[0]
print(actions,states)

def build_model(states,actions):
  model=Sequential()
  #model.add(Embedding(states,10, input_length=1))
  #model.add(Reshape((10,)))
  model.add(Flatten(input_shape=(1,states)))
  model.add(Dense(32,activation='relu'))
  model.add(Dense(32,activation='relu'))
  model.add(Dense(actions,activation='linear'))
  return model
  

4 8


In [4]:
import rl
from rl.agents import DQNAgent
from rl.policy import EpsGreedyQPolicy,BoltzmannQPolicy
from rl.memory import SequentialMemory
def build_model(states,actions):
  model=Sequential()
  #model.add(Embedding(states,10, input_length=1))
  #model.add(Reshape((10,)))
  model.add(Flatten(input_shape=(1,states)))
  model.add(Dense(32,activation='relu'))
  model.add(Dense(32,activation='relu'))
  model.add(Dense(actions,activation='linear'))
  return model

model1=build_model(states,actions)
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 8)                 0         
                                                                 
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 4)                 132       
                                                                 
Total params: 1,476
Trainable params: 1,476
Non-trainable params: 0
_________________________________________________________________


In [5]:
#from rl.agents.sarsa import EpsGreedyQPolicy
policy=EpsGreedyQPolicy()
memory=SequentialMemory(limit=50000,window_length=1)

agent=DQNAgent(model=model1,memory=memory,policy=policy,nb_actions=actions,nb_steps_warmup=500, target_model_update=1e-2)

In [6]:
agent.compile(Adam(lr=0.001),metrics=['mae'])
agent.fit(env, nb_steps=500000, visualize=False, verbose=1)

 3807/10000 [==========>...................] - ETA: 1:27 - reward: -0.1442done, took 743.693 seconds


In [ ]:
env=wrappers.RecordVideo(env,'videos1',episode_trigger=lambda x: x%1==0)
scores=agent.test(env,nb_episodes=10,visualize=False)

In [ ]:
#save the model weights
agent.save_weights('dqn_weights.h5f',overwrite=True)